# S02E03

In [ ]:
from dotenv import load_dotenv
from openai import OpenAI
import os
import httpx

load_dotenv()

In [ ]:
centrala = os.environ.get("AG3NTS_CENTRALA_URL")
key = os.environ.get("AG3NTS_API_KEY")

url = f"{centrala}/data/{key}/robotid.json"

In [ ]:
def get_content(url):
    try:
        response = httpx.get(url)
        response.raise_for_status()
        payload = response.json()
        return payload["description"]
    except Exception as e:
        print(e)
        return None

In [ ]:
description = get_content(url)

In [ ]:
client = OpenAI()

In [ ]:
meta_prompt = f"""
Twoim zadaniem jest zbudowanie prompt służącego do generowania obrazu przez model DALL-E-3 
w oparciu o podane przez użytkowanika informacje zawierające opis robota.
Przeanalizuj informacje i stwórz odpowiedni prompt zawierający najbardziej istotne cechy robota.
Skup się na wyglądzie. Pomiń informacje, które nie mają wpływu na wygląd robota.
"""

engineered_prompt = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": meta_prompt},
        {"role": "user", "content": description},
    ],
).choices[0].message.content

In [ ]:
generation = client.images.generate(
    model="dall-e-3",
    prompt=engineered_prompt,
    size="1024x1024",
    quality="standard",
    n=1
)

image_url = generation.data[0].url

In [ ]:
image_url

In [ ]:
from aidevs3.poligon import send

verification_url = f"{centrala}/report"
key = os.environ.get("AG3NTS_API_KEY")

res = send(verification_url, task="robotid", apikey=key, answer=image_url)